In [8]:
import os
from dotenv import load_dotenv
import geopandas as gpd
from shapely.geometry import Polygon, mapping
import requests 
import json
import time
from requests.auth import HTTPBasicAuth
import rasterio 
import matplotlib.pyplot as plt 
from PIL import Image

In [9]:
load_dotenv()
PLANET_API_KEY = os.getenv('API_KEY')

In [10]:

"""
This script reads all shapefiles in a directory, and extracts the AOI polygons 
stored in the shapefile to a geopandas Polygon to query Planet Imagery downloads
through their API. The code logic assumes only one polygon is stored in each
shapefile.

Harrison Myers
6/7/2024

Dependencies: 
    -- os
    -- geopandas (pip install geopandas)
    -- shapely (pip install shapely)
"""
import geopandas as gpd
from shapely.geometry import Polygon
from pyproj import Transformer
import os

dir_path = '/Users/aidanhayes/Desktop/Planet-Imagery-API/PlanetImageryAOIs'
os.chdir(dir_path)

def transform_coordinates(easting, northing, from_epsg, to_epsg):
    """
    Transforms easting-northing coordinates to lon/lat
    
    Params:
        easting (float, req): easting location
        northing (float, req): northing location
        from_epsg (int, req): EPSG code for spatial projection easting and 
                              northing points are in
        to_epsg (int, req): EPSG code to convert to for lat/lon
        
    Returns:
        lon, lat (float): longitude and latitude of easting/northing coordinate
    """
    transformer = Transformer.from_crs(f'epsg:{from_epsg}', f'epsg:{to_epsg}', always_xy=True)
    lon, lat = transformer.transform(easting, northing)
    return lon, lat

def getPolygons(folderPath, from_epsg, to_epsg):
    """
    Reads all shapefiles in a directory and stores their respective polygons
    in a dictionary
    Params:
        -- folderPath (str, req): Path to folder where shapefiles are stored
    returns:
        -- polygons (dict): Dictionary of polygons indexed by first two letters
                            of site name
    """
    polygons = {}
    for f in os.listdir(folderPath):
        if f.endswith('.shp'): # only read shapefiles
            fpath = os.path.join(folderPath, f) # get path to shapefile
            gdf = gpd.read_file(fpath) # read shapefile
            for geom in gdf.geometry:
                if isinstance(geom, Polygon):
                    coords = geom.exterior.coords._coords[:, :2]
                    lonlat_coords = [transform_coordinates(x, y, from_epsg, to_epsg) for x, y in coords]
                    polygons[f'{f[:3]}'] = lonlat_coords
    return polygons
                    
polygon_dict = getPolygons(dir_path, 32145, 4326)

            

In [11]:
# Making sure our API KEY is working 
BASE_URL = 'https://api.planet.com/tasking/v2/orders/'
auth = HTTPBasicAuth(PLANET_API_KEY, '')
res = requests.get(url=BASE_URL, auth=auth)
print(res.status_code)

200


In [12]:
# function definition for searching the stats of each image site 
def search_params_stats(coordinates, DateTime):
    
    return {
    "item_types":[
        "PSScene"
    ],
    "interval": "year",
    "filter":{
        "type":"AndFilter",
        "config":[
            {
                "type":"GeometryFilter",
                "field_name":"geometry",
                "config": {
                    'type':'Polygon',
                    'coordinates':[coordinates]
                }
                        
            },
            {
                "type":"DateRangeFilter",
                "field_name":"acquired",
                "config":{
                "gte":DateTime,
                }
            },
            {
                "type":"StringInFilter",
                "field_name":"quality_category",
                "config":[
                "standard"
                ]
            },
            {
                "type":"AssetFilter",
                "config":[
                "ortho_analytic_8b_sr"
                ]
            },
            {
                "type":"RangeFilter",
                "field_name":"cloud_cover",
                "config":{
                "lte":0.10
                },
            },
            {
                "type":"RangeFilter",
                "field_name":"visible_percent",
                "config":{
                "gte":.95
                },
            },
            {
                "type":"PermissionFilter",
                "config":[
                "assets:download"
                ]
            },
        ]
    }
    }


In [13]:
# function definition to grab the image ids for ordering
def search_params(coordinates, DateTime):
    
    return {
    "item_types":[
        "PSScene"
    ],
    "filter":{
        "type":"AndFilter",
        "config":[
            {
                "type":"GeometryFilter",
                "field_name":"geometry",
                "config": {
                    'type':'Polygon',
                    'coordinates':[coordinates]
                }
                        
            },
            {
                "type":"DateRangeFilter",
                "field_name":"acquired",
                "config":{
                "gte":DateTime,
                }
            },
            {
                "type":"StringInFilter",
                "field_name":"quality_category",
                "config":[
                "standard"
                ]
            },
            {
                "type":"AssetFilter",
                "config":[
                "ortho_analytic_8b_sr"
                ]
            },
            {
                "type":"RangeFilter",
                "field_name":"cloud_cover",
                "config":{
                "lte":0.10
                },
            },
            {
                "type":"RangeFilter",
                "field_name":"visible_percent",
                "config":{
                "gte":.95
                },
            },
            {
                "type":"PermissionFilter",
                "config":[
                "assets:download"
                ]
            },
        ]
    }
    }


In [14]:
polygon_dict.keys()

dict_keys(['Mon', 'Pay', 'Des', 'QHP', 'QBP'])

In [15]:
# Stat Searchup API Call
SEARCH_ENDPOINT = 'https://api.planet.com/data/v1/stats'
headers = {
    'Authorization': f'api-key {PLANET_API_KEY}',
    'Content-Type': 'application/json'
}

lst_of_daterangefilters = ["2021-11-18T00:00:00Z", "2021-11-15T00:00:00Z", "2021-11-15T00:00:00Z", "2019-09-27T00:00:00Z", "2019-08-27T00:00:00Z"]

i = 0

for key in polygon_dict:
    geojson_geometry = polygon_dict[key]
    coordinates = [list(coord) for coord in geojson_geometry]
    search_parameters = search_params_stats(coordinates, lst_of_daterangefilters[i])
    i += 1
    response = requests.post(SEARCH_ENDPOINT, headers=headers, data=json.dumps(search_parameters))
    data_dict = json.loads(response.text)
    total = 0
    for lst in data_dict['buckets']:
        total += lst['count']
    print(f'{key} site total images: {total}')

Mon site total images: 414
Pay site total images: 405
Des site total images: 413
QHP site total images: 523
QBP site total images: 513


In [18]:
# Gets the Image IDS in batches of 250 (or less)
# Ran different times for allowed maximum download batch of API
SEARCH_ENDPOINT = 'https://api.planet.com/data/v1/quick-search'
image_ids = []
curr_key = 'QBP'

geojson_geometry = polygon_dict[curr_key]
coordinates = [list(coord) for coord in geojson_geometry]
search_parameters = search_params(coordinates, "2019-08-27T00:00:00Z")
response = requests.post(SEARCH_ENDPOINT, headers=headers, data=json.dumps(search_parameters))
data_dict = json.loads(response.text)

def process_response(data_dict):
    if 'features' in data_dict:
        for feature in data_dict['features']:
            image_id = feature['id']
            image_ids.append(image_id)
    else:
        print("No features found in response.")

process_response(data_dict)

In [19]:
# Ran different times for allowed maximum download batch of API
# Gets next page of Image IDs
def handle_pagination(data_dict):
    while '_links' in data_dict and '_next' in data_dict['_links']:
        next_url = data_dict['_links']['_next']
        if not next_url:
            print("No next URL found - stopping")
            break
        print(f"Fetching next URL: {next_url}")
        retry_attempts = 0
        while True:
            response = requests.get(next_url, headers=headers)
            if response.status_code == 200:
                data_dict = response.json()
                process_response(data_dict)
                break
            elif response.status_code == 429:
                retry_attempts += 1
                wait_time = 2 ** retry_attempts  # Exponential backoff
                print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                print(f"Error: Received status code {response.status_code} while fetching next URL")
                print(response.text)
                response.raise_for_status()

handle_pagination(data_dict)

print(f"Total image IDs retrieved: {len(image_ids)}")

Fetching next URL: https://api.planet.com/data/v1/searches/716710efda52437183306ab16a6f652d/results?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogInB1Ymxpc2hlZCIsICJzb3J0X2Rlc2MiOiB0cnVlLCAic29ydF9zdGFydCI6ICIyMDIyLTExLTIwVDA0OjI3OjQ5LjAwMDAwMFoiLCAic29ydF9sYXN0X2lkIjogIjIwMjIxMTE5XzE1MjUyMF8zMV8yNDllIiwgInNvcnRfcHJldiI6IGZhbHNlLCAicXVlcnlfcGFyYW1zIjoge319
Fetching next URL: https://api.planet.com/data/v1/searches/716710efda52437183306ab16a6f652d/results?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogInB1Ymxpc2hlZCIsICJzb3J0X2Rlc2MiOiB0cnVlLCAic29ydF9zdGFydCI6ICIyMDIwLTA5LTIwVDAxOjE2OjUwLjAwMDAwMFoiLCAic29ydF9sYXN0X2lkIjogIjIwMjAwOTE5XzE1MDAzMV84MF8yMzA0IiwgInNvcnRfcHJldiI6IGZhbHNlLCAicXVlcnlfcGFyYW1zIjoge319
Fetching next URL: https://api.planet.com/data/v1/searches/716710efda52437183306ab16a6f652d/results?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogInB1Ymxpc2hlZCIsICJzb3J0X2Rlc2MiOiB0cnVlLCAic29ydF9zdGFydCI6ICIyMDIwLTA4LTA2VDIzOjAyOjU5LjAwMDAwMFoiLCAic29ydF9sYXN0X2lkIjogIjIwMjAwODA2

In [23]:
SEARCH_ENDPOINT = 'https://api.planet.com/compute/ops/orders/v2'
# search_parameters = {
#     "name": "Aidan-test-20",
#     "source_type": "scenes",
#     "products": [
#         {
#             "item_ids": image_ids,
#             "item_type": "PSScene",
#             "product_bundle": "analytic_8b_sr_udm2"
#         }
#     ],
#     "tools": [
#         {
#             "clip": {
#                 "aoi": {
#                     'type': 'Polygon',
#                     'coordinates': [coordinates]
#                 }
#             }
#         }
#     ],
#     "order": [
#         {
#             "name": f"composite-{i}",
#             "source_type": "scenes",
#             "products": [
#                 {
#                     "item_ids": [image_ids[i]],
#                     "item_type": "PSScene",
#                     "product_bundle": "analytic_8b_sr_udm2"
#                 }
#             ],
#             "tools": [
#                 {
#                     "clip": {
#                         "aoi": {
#                             'type': 'Polygon',
#                             'coordinates': [coordinates]
#                         }
#                     }
#                 },
#                 {
#                     "composite": {}
#                 }
#             ]
#         }
#         for i in range(len(image_ids))
#     ],
#     "delivery": {
#         "single_archive": True,
#         "archive_type": "zip"
#     }
# }

# response = requests.post(SEARCH_ENDPOINT, headers=headers, data=json.dumps(search_parameters))

# if response.status_code == 200:
#     print("Request was successful.")
#     print(response.json())
# else:
#     print("Request failed.")
#     print(response.status_code)
#     print(response.text)

In [24]:
# getting the order ID and the order url 
# order_id = response.json()['id']
order_id = 'f31be135-2185-49f5-bc49-fdeaa288c0ea'
print(f'Order ID: {order_id}')
order_url = f"{SEARCH_ENDPOINT}/{order_id}"
response = requests.get(order_url, headers=headers)

Order ID: f31be135-2185-49f5-bc49-fdeaa288c0ea


In [42]:
# # making directory if not already there
# folder_destination = f'metadata_{curr_key}'
# try:
#     os.mkdir(folder_destination)
# except OSError as error:
#     print(error)


# zip_name_output = f'{order_id}/output.zip'
# # print(zip_name_output)
# # storing all metadata in folder
# response_json = response.json()
# for delivery in response_json['_links']['results']:
#     # print(delivery['name'])
#     if delivery['name'] == zip_name_output:
#         link = delivery['location']
#         zip_response = requests.get(link, stream=True)
#         zip_file_path = os.path.join(folder_destination, 'output.zip')
#         with open(zip_file_path, 'wb') as file:
#                 for chunk in zip_response.iter_content(chunk_size=8192):
#                     file.write(chunk)
                    
#         print(f"Zip file has been downloaded and saved to {zip_file_path}")
#     else:
#         print("Request failed.")
#         print(response.status_code)
#         print(response.text)


[Errno 17] File exists: 'metadata_QBP'
Zip file has been downloaded and saved to metadata_QBP/output.zip
Request failed.
200
{"_links":{"_self":"https://api.planet.com/compute/ops/orders/v2/f31be135-2185-49f5-bc49-fdeaa288c0ea","results":[{"delivery":"success","expires_at":"2024-06-19T15:54:25.854Z","location":"https://api.planet.com/compute/ops/download/?token=eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE3MTg4MTI0NjUsInN1YiI6Ilh1cXJlbE5vV0tyRHRkdERlYlJkaXQ1bVBFalNnVWN1dUlOd1FRcDdacHMzRDNiQzJpWHl1QjVyUmxzSmYxRmcwdHhMdTBrS2hteWZSQXNqa0VQeFlRPT0iLCJ0b2tlbl90eXBlIjoiZG93bmxvYWQtYXNzZXQtc3RhY2siLCJhb2kiOiIiLCJhc3NldHMiOlt7Iml0ZW1fdHlwZSI6IiIsImFzc2V0X3R5cGUiOiIiLCJpdGVtX2lkIjoiIn1dLCJ1cmwiOiJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY29tcHV0ZS1vcmRlcnMtbGl2ZS9mMzFiZTEzNS0yMTg1LTQ5ZjUtYmM0OS1mZGVhYTI4OGMwZWEvb3V0cHV0LnppcD9YLUdvb2ctQWxnb3JpdGhtPUdPT0c0LVJTQS1TSEEyNTZcdTAwMjZYLUdvb2ctQ3JlZGVudGlhbD1jb21wdXRlLWdjcy1zdmNhY2MlNDBwbGFuZXQtY29tcHV0ZS1wcm9kLmlhbS5nc2VydmljZWFjY291bnQuY29tJTJGMjA

In [45]:
for i in range(100):
    i += 1
    if i % 3 == 0 | i % 5 == 0:
        print("FizzBuzz")
    elif i % 3 == 0:
        print('Fizz')
    elif i % 5 == 0:
        print("Buzz")
    else:
        print(i)

1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14
FizzBuzz
16
17
Fizz
19
Buzz
Fizz
22
23
Fizz
Buzz
26
Fizz
28
29
FizzBuzz
31
32
Fizz
34
Buzz
Fizz
37
38
Fizz
Buzz
41
Fizz
43
44
FizzBuzz
46
47
Fizz
49
Buzz
Fizz
52
53
Fizz
Buzz
56
Fizz
58
59
FizzBuzz
61
62
Fizz
64
Buzz
Fizz
67
68
Fizz
Buzz
71
Fizz
73
74
FizzBuzz
76
77
Fizz
79
Buzz
Fizz
82
83
Fizz
Buzz
86
Fizz
88
89
FizzBuzz
91
92
Fizz
94
Buzz
Fizz
97
98
Fizz
Buzz
